In [ ]:
%env SERVER_URL    127.0.0.1
%env PROXY_SUBURL  proxy/7474/browser/

In [47]:
!pip3 install ipython-cypher neo4j py2neo >/dev/null

In [48]:
%env NEO4J_VERSION neo4j-community-4.1.1
%env NEO4J_PATH    neo4j-community-4.1.1

env: NEO4J_VERSION=neo4j-community-4.1.1
env: NEO4J_PATH=neo4j-community-4.1.1


In [49]:
!${NEO4J_PATH}/bin/neo4j status

Neo4j is not running


In [50]:
!curl http://localhost:7474/

{
  "bolt_routing" : "neo4j://localhost:7687",
  "transaction" : "http://localhost:7474/db/{databaseName}/tx",
  "bolt_direct" : "bolt://localhost:7687",
  "neo4j_version" : "4.1.1",
  "neo4j_edition" : "community"
}

In [51]:
# criando um adapter e verificando a conexão
from py2neo import Graph

graph = Graph("bolt://127.0.0.1:7687")

print(dir(graph))

graph.delete_all()
print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.relationships)))



['__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_procedures', 'auto', 'begin', 'call', 'create', 'delete', 'delete_all', 'evaluate', 'exists', 'match', 'match_one', 'merge', 'name', 'node_cache', 'nodes', 'play', 'pull', 'push', 'relationship_cache', 'relationships', 'run', 'schema', 'separate', 'service']
Number of nodes: 0 
Number of relationships: 0 


In [54]:
!cp -Rf ./dataset ${NEO4J_PATH}/import/

In [52]:
graph.run('CALL apoc.help("apoc")')

 type      | name                     | text                                                                                                                                                                                             | signature                                                                                                                                                                                                                    | roles | writes | core 
-----------|--------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------|--------|------
 procedure | apoc.algo.aStar          

In [56]:
command = """CALL apoc.load.json('dataset/olist_products_dataset.json')"""

graph.run(command)

 value                                                                                                                                                                                                                                                               
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 {product_category_name: 'perfumaria', product_height_cm: 10, product_id: '1e9e8ef04dbcff4541ed26657ea517e5', product_weight_g: 225, product_photos_qty: 1, product_width_cm: 14, product_name_lenght: 40, product_description_lenght: 287, product_length_cm: 16}   
 {product_category_name: 'artes', product_height_cm: 18, product_id: '3aa071139cb16b67ca9e5dea641aaa2f', product_weight_g: 1000, product_photos_qty: 1, product_width_cm: 20, product_name_lenght: 44, product_descrip

In [58]:
print(dir(graph))

['__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_procedures', 'auto', 'begin', 'call', 'create', 'delete', 'delete_all', 'evaluate', 'exists', 'match', 'match_one', 'merge', 'name', 'node_cache', 'nodes', 'play', 'pull', 'push', 'relationship_cache', 'relationships', 'run', 'schema', 'separate', 'service']
